This project is based on the MAGIC Gamma Telescope Data Set from the UCI Machine Learning Repository. The data set contains 19020 gamma-ray observations and 12332 hadron observations. The goal is to classify the observations as either gamma-ray or hadron. The data set contains 11 features, which are described below. The data set is available at https://archive.ics.uci.edu/ml/datasets/MAGIC+Gamma+Telescope. It follows along with the "Machine Learning for Everyone" video from FreeCodeCamp.org. The video can be found at https://www.youtube.com/watch?v=7eh4d6sabA0. At certain sections of the project additional sections are added to explain the code and the concepts behind it. Also certain sections might have added code and features which are not in the original video. The original video is a great introduction to ML and is highly recommended. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

: 

Data Set Information:

The data are MC generated (see below) to simulate registration of high energy gamma particles in a ground-based atmospheric Cherenkov gamma telescope using the imaging technique. Cherenkov gamma telescope observes high energy gamma rays, taking advantage of the radiation emitted by charged particles produced inside the electromagnetic showers initiated by the gammas, and developing in the atmosphere. This Cherenkov radiation (of visible to UV wavelengths) leaks through the atmosphere and gets recorded in the detector, allowing reconstruction of the shower parameters. The available information consists of pulses left by the incoming Cherenkov photons on the photomultiplier tubes, arranged in a plane, the camera. Depending on the energy of the primary gamma, a total of few hundreds to some 10000 Cherenkov photons get collected, in patterns (called the shower image), allowing to discriminate statistically those caused by primary gammas (signal) from the images of hadronic showers initiated by cosmic rays in the upper atmosphere (background).

Typically, the image of a shower after some pre-processing is an elongated cluster. Its long axis is oriented towards the camera center if the shower axis is parallel to the telescope's optical axis, i.e. if the telescope axis is directed towards a point source. A principal component analysis is performed in the camera plane, which results in a correlation axis and defines an ellipse. If the depositions were distributed as a bivariate Gaussian, this would be an equidensity ellipse. The characteristic parameters of this ellipse (often called Hillas parameters) are among the image parameters that can be used for discrimination. The energy depositions are typically asymmetric along the major axis, and this asymmetry can also be used in discrimination. There are, in addition, further discriminating characteristics, like the extent of the cluster in the image plane, or the total sum of depositions.

The data set was generated by a Monte Carlo program, Corsika, described in:
D. Heck et al., CORSIKA, A Monte Carlo code to simulate extensive air showers,
Forschungszentrum Karlsruhe FZKA 6019 (1998).
[Web Link]

In [ ]:
cols=['fLength','fWidth','fSize','fConc','fConc1','fAsym','fM3Long','fM3Trans','fAlpha','fDist','class']
df = pd.read_csv("magic04.data", names=cols)
df.head()

: 

In [ ]:
df["class"].unique()

: 

We can see we have 2 main types of classes: g(gammas) and g(hadrons). But since computers are better at understanding numbers than letters we will convert these two classes to numbers (0,1) by converting them into integers: 

In [ ]:
df["class"] = df["class"].map({"g":1, "h":0})

: 

If we run the code that gives us the array of our unique classes again we now see that these are now composed of zeros and ones: 

In [ ]:
df["class"].unique()

: 

In [ ]:
df.head()

: 

What we can do next is, for each data feature (represented in the for loop with "label") plot a histogram that will allow us to quickly visualize our data and see if there are any obvious correlations between certain range of values of our features and the particle class. 

In [ ]:
for label in cols[:-1]:
    plt.hist(df[df["class"]==1][label], alpha=0.7, label="gamma", density=True)
    plt.hist(df[df["class"]==0][label], alpha=0.7, label="hadron", density=True)
    plt.title(label)
    plt.ylabel("Probability")
    plt.legend()
    plt.show()


: 

Next we are going to build our train, validation and test sets. We will use 60% of our data for training, 20% for validation and 20% for testing.

In [ ]:
train, valid, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])

: 

If we look at the different scales of our features we can see that they are not all on the same scale. This can be problematic for our model since it will be difficult for it to learn the weights of the different features. 

Optimization algorithms are used to train machine learning models by finding the set of weights that minimize the error of the model on the training data. These algorithms use mathematical techniques to iteratively adjust the weights in order to minimize the error.

Optimization algorithms are sensitive to the scale of the features because the error function used to evaluate the model's performance is often based on the difference between the predicted values and the true values. If the scales of the features are significantly different, then the error function may be dominated by the features with larger scales, even if those features are not as important for predicting the target variable. Since higher scale features will have larger absolute differences between the predicted values and the true values. 

For example, consider a model that is trying to predict the price of a house based on the size of the house (measured in square feet) and the number of bedrooms. If the size of the house is measured in the thousands of square feet, but the number of bedrooms is only measured in the single digits, then the scale of the "size" feature will be much larger than the scale of the "bedrooms" feature. As a result, the error function may be dominated by the "size" feature, even if the "bedrooms" feature is actually more important for predicting the price of the house.

This can lead to suboptimal model performance, because the model is weighting the features incorrectly. In order to address this issue, it is often helpful to scale the features so that they are on a similar scale before training the model. This can be done using techniques such as min-max scaling or standardization. Scaling the features can help the model to more accurately estimate the relative importance of the different features, which can improve model performance.

To do this from sklearn.preprocessing we import StandardScaler and then we create a scaler object, and then we fit the scaler to our data. 

In [ ]:
def scale_dataset(dataframe, oversample=False):
    x = dataframe[dataframe.columns[:-1]].values
    y = dataframe[dataframe.columns[-1]].values

    scaler = StandardScaler()
    x = scaler.fit_transform(x)

    if oversample:
        ros = RandomOverSampler()
        x, y = ros.fit_resample(x, y)

    data = np.hstack((x, np.reshape(y, (-1,1))))
    return data, x, y

: 

We should now look at how many occurrences of each class we have in our training set. 

In [ ]:
print(len(train[train["class"]==1])) #gamma
print(len(train[train["class"]==0])) #hadron

: 

Since there is a big difference in the number of occurrences of each class we should consider using oversampling as to not bias our model towards the class with the most occurrences. To to this we added the oversampling parameter to our scaler_dataset function (above).

What the oversample function does is it takes the minority class (hadrons) and randomly duplicates it until it has the same number of occurrences as the majority class (gammas).

By default we put the oversample to False. But when we call the train we will set it to True. Hence the "if oversample:" statement.

In [ ]:
#train, x_train, y_train = scale_dataset(train, oversample=True)

: 

Let's double check the number of occurrences of each class in our training set after oversampling.

In [ ]:
#print(len(y_train))
#print(sum(y_train == 1)) #gamma
#print(sum(y_train == 0)) #hadron

#these commands were used only to verify that the oversampling worked, that is why they are commented out and I just left the output below. 

: 

Total Number of Classifications: 14756;
Total Occurrences in Class 1: 7378;
Total Occurrences in Class 0: 7378;

Indeed we see that the number of occurrences of each class is now the same and we can have confidence that our model will not be biased towards a class. 

Below the oversample parameter is set to True for the first call to scale_dataset to apply oversampling to the training data, and set to False for the last two calls to avoid oversampling the validation and test data. This is done to ensure that the validation and test sets are representative of the true distribution of the data, and to avoid artificially inflating the performance of the model on those sets.

In [ ]:

train, x_train, y_train = scale_dataset(train, oversample=True)
valid, x_valid, y_valid = scale_dataset(valid, oversample=False)
test, x_test, y_test = scale_dataset(test, oversample=False)


: 

Now that we have our data ready we will go through several different ML models and apply them to our data. We will go through KNN, Naive Bayes, Logistic Regression, SVM, Neural Networks (NN), Classification NN using Tensor Flow, Linear Regression, Linear Regression NN using Tensor Flow, K-means Clustering, and finally Principal Component Analysis (PCA). But first let's start with a simple model: KNN:

KNN: K-Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

: 

Next we will use the KNN algorithm to classify our data and see how well it performs by printing out the classification report. We can try out different amounts of neighbors and see how the accuracy changes. But after a few manual iterations I see we seem to get the best accuracy when we use 13 neighbors. We reached an accuracy of 0.83%. Also keep in mind that more neighbhors means more computation time at diminishing returns. There is not a big difference (relative) in terms of accuracy score between 13 and 5 neighbors (0.01% difference). Note, you should use an odd number of neighbors to avoid ties.

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=13)
knn_model.fit(x_train, y_train)

: 

In [ ]:
y_pred = knn_model.predict(x_test)
print(classification_report(y_test, y_pred, target_names=["hadron", "gamma"]))

: 

But what if there is another K value with a better accuracy score? How can we make an automated way to find the best amount of neighbors? 
We can use a for loop to iterate through different amounts of neighbors and see which one gives us the best accuracy.
And then we represent that iteration in a graph using matplotlib's pyplot.

In [ ]:
k_values = range(1, 50)
acc_score = []
for i in k_values:
    knn_model = KNeighborsClassifier(n_neighbors=i)
    knn_model.fit(x_train, y_train)
    y_pred = knn_model.predict(x_test)
    acc_score.append(knn_model.score(x_test, y_test))

optimal_k = k_values[acc_score.index(max(acc_score))]

plt.plot(k_values, acc_score, color='blue',linestyle='dashed', marker='o')
plt.figsize = (10, 6)
plt.xlabel("K")
plt.ylabel("Accuracy")
plt.title("KNN Accuracy")
plt.suptitle(f"The optimal value of K is {optimal_k} with an accuracy of {max(acc_score):.2f}%.", y=1.05)
plt.axvline(optimal_k, color='green', linewidth=2)
plt.annotate(f"{optimal_k}", xy=(optimal_k, max(acc_score)), xytext=(optimal_k + 0.5, max(acc_score)))
plt.show()


: 

Now that we have our best K value we can use it to train our model and then print out the classification report.

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=optimal_k)
knn_model.fit(x_train, y_train)


: 

In [ ]:
y_pred = knn_model.predict(x_test)
print(classification_report(y_test, y_pred, target_names=["hadron", "gamma"]))


: 

Let's dissect the classification report: 
- The first column is the precision of the model. Precision is the number of true positives divided by the number of true positives plus the number of false positives. The higher the precision score the lowest the number of false positives. In other terms, how many of the ones(zeros) we labeled as ones(zeros) were actually ones(zeros).
- The second column is the recall of the model. Recall is the number of true positives divided by the number of true positives plus the number of false negatives. The higher the recall score the lowest the number of false negatives. In simpler terms, how many of the ones(zeros) did we label as ones(zeros).
- The third column is the f1-score of the model. The f1-score is the harmonic mean of the precision and recall. The higher the f1-score the better the model.
- The fourth column is the support of the model. Support is the number of occurrences of each class in the test set.
- The accuracy of the model is the average of the precision, recall, and f1-score for each class.
- The macro average is the average of the metric computed for each class
- The weighted average is the average of the metric, weighted by the number of true instances for each class.
- The support is the number of occurrences of each class in the test set.


For example, consider a classification report with two classes, A and B. The precision, recall, and f1-score for class A are 0.8, 0.7, and 0.75, respectively, and the precision, recall, and f1-score for class B are 0.6, 0.9, and 0.72, respectively. In this case, the macro average for precision would be (0.8 + 0.6) / 2 = 0.7, the macro average for recall would be (0.7 + 0.9) / 2 = 0.8, and the macro average for f1-score would be (0.75 + 0.72) / 2 = 0.735.

On the other hand, if class A has 10 true instances and class B has 5 true instances, the weighted average for precision would be (0.8 * 10 + 0.6 * 5) / 15 = 0.76, the weighted average for recall would be (0.7 * 10 + 0.9 * 5) / 15 = 0.78, and the weighted average for f1-score would be (0.75 * 10 + 0.72 * 5) / 15 = 0.737.

The choice between using the macro average or the weighted average in a classification report depends on the specific context and the goals of the evaluation. The macro average is useful when you want to balance the metric across all classes, while the weighted average is useful when you want to give more weight to classes with more instances.

And that's a wrap for KNN. Let's move on to Naive Bayes:

Naive Bayes

[label](../../../../Desktop/Screenshot%202022-12-26%20at%2015.29.43.png%0D) ![Alt text](../../../../Desktop/Screenshot%202022-12-26%20at%2015.29.54.png)

To understand Naive Bayes we need to understand Bayes Theorem. Bayes Theorem is a way to calculate the probability of an event based on prior knowledge of conditions that might be related to the event. Bayes Theorem is stated mathematically as the following equation:
P(A|B) = P(B|A) * P(A) / P(B)

Where P(A|B) is a conditional probability: the likelihood of event A occurring given that B is true. P(B|A) is also a conditional probability: the likelihood of event B occurring given that A is true. P(A) and P(B) are the probabilities of observing A and B respectively.

I recommend you see this video by Krish Naik (a great tutor of Data Science btw) to have a better understanding of Bayes Theorem: https://www.youtube.com/watch?v=jS1CKhALUBQ&ab_channel=KrishNaik

The code to implement this algorithm is very simple. We just need to call the GaussianNB function from the sklearn library and then call the fit function to train our model. Then we can print out the classification report.

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb_model = GaussianNB()
nb_model = nb_model.fit(x_train, y_train)


: 

In [ ]:
y_pred = nb_model.predict(x_test)
print(classification_report(y_test, y_pred, target_names=["hadron", "gamma"]))

: 

As you can see our accuracy score is lower than the classification report we got from KNN. Let's move on to Logistic Regression:

Logistic Regression